In [1]:
"""
Script to test the SAMRAI data reader for data from all levels.
"""

from floatpy.readers import samrai
import numpy

%matplotlib inline
from matplotlib import pyplot as plt
plt.rc("savefig", dpi = 150)


In [2]:
"""
Set the path of the data directory for the data reader.
"""

data_reader = samrai.samraiDataReader()
data_reader.setDataDirectoryPath('/mnt/b9e17de4-9838-4425-ad4c-dc2ad3e41075/manlong/HAMeRS/tests/' + \
                                 '2D_Poggi_Richtmyer_Meshkov_instability/2D_Poggi_Richtmyer_Meshkov_instability_1/' + \
                                 'viz_2D_Poggi_Richtmyer_Meshkov_instability/visit_dump.12002/')


In [3]:
"""
Try to read the full domain shape refined to the highest level. 
"""

data_reader.readSummary()

domain_shape = data_reader.getRefinedDomainSize()

print domain_shape


[5000  500]


In [4]:
"""
Try to read the density and velocity data from all levels.
"""

var_names = ['density', 'velocity']
num_ghosts = numpy.array([0, 4], dtype = numpy.int)
periodic_dimension = numpy.array([False, True], dtype = numpy.bool)

data_reader.readCombinedDataFromAllLevels(var_names, num_ghosts, periodic_dimension)

x_coords, y_coords, z_coords = data_reader.getDataCoordinates()
data_rho = data_reader.getData('density')
data_vel = data_reader.getData('velocity')

print data_rho.shape
print data_vel.shape


(1, 5000, 508)
(2, 5000, 508)


In [5]:
"""
Check the values of the ghost cells.
"""

print data_rho[0, 0, 0:4]
print data_rho[0, 0, -4*2:-4]

print data_rho[0, 0, -4:]
print data_rho[0, 0, 4:8]


[ 12.74021721  12.74021721  12.74021721  12.74021721]
[ 12.74021721  12.74021721  12.74021721  12.74021721]
[ 12.74021721  12.74021721  12.74021721  12.74021721]
[ 12.74021721  12.74021721  12.74021721  12.74021721]


In [1]:
"""
Plot the level of data. (NAN for non-existing data)
"""

data_masked = numpy.ma.masked_where(numpy.isnan(data_rho), data_rho)
X, Y = numpy.meshgrid(x_coords, y_coords)

plt.pcolormesh(X, Y, data_masked[0,:,:].T)
plt.axes().set_aspect('equal', 'datalim')
plt.xlabel('x')
plt.ylabel('y')
plt.show()


NameError: name 'numpy' is not defined